In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../Dataset/api_data_aadhar_enrolment/aadhar_enrolment_final.csv')

In [4]:
data.columns

Index(['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17',
       'age_18_greater'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006029 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date            1006029 non-null  object
 1   state           1006029 non-null  object
 2   district        1006029 non-null  object
 3   pincode         1006029 non-null  int64 
 4   age_0_5         1006029 non-null  int64 
 5   age_5_17        1006029 non-null  int64 
 6   age_18_greater  1006029 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 53.7+ MB


In [11]:
data.shape

(1006029, 7)

In [14]:
data.duplicated().value_counts()

False    983072
True      22957
Name: count, dtype: int64

In [24]:
data.duplicated(
    subset=[
        'date','state','district','pincode',
        'age_0_5','age_5_17','age_18_greater'
    ]
).value_counts()

False    983072
True      22957
Name: count, dtype: int64

In [19]:
(data['state']=="100000").sum()

np.int64(22)

# District Report

In [ ]:
import re

district_view = (
    data[['state', 'district']]
    .dropna()
    .assign(
        district_norm=lambda df:
            df['district']
            .str.lower()
            .str.strip()
            .str.replace(r'[()\-\s]+', ' ', regex=True)
    .str.replace(r'[^a-z0-9 ]', '', regex=True)
    )
)


In [12]:
variation_summary = (
    district_view
    .groupby(['state', 'district_norm'])['district']
    .nunique()
    .reset_index(name='no_of_variations')
    .query('no_of_variations > 1')
)


In [13]:
district_variations = (
    district_view
    .merge(
        variation_summary[['state', 'district_norm']],
        on=['state', 'district_norm']
    )
    .groupby(['state', 'district_norm'])['district']
    .unique()
    .reset_index(name='district_spellings')
)


In [14]:
district_variations

,state,district_norm,district_spellings
0,Bihar,aurangabadbh,"[Aurangabad(bh), Aurangabad(BH)]"
1,Chhattisgarh,janjgir champa,"[Janjgir - Champa, Janjgir Champa]"
2,Jammu and Kashmir,punch,"[Punch, punch]"
3,Jharkhand,seraikelakharsawan,"[Seraikela-Kharsawan, Seraikela-kharsawan]"
4,Karnataka,yadgir,"[Yadgir, yadgir]"
5,Odisha,angul,"[Angul, ANGUL]"
6,Odisha,anugul,"[Anugul, ANUGUL]"
7,Odisha,jajpur,"[JAJPUR, jajpur, Jajpur]"
8,Odisha,nuapada,"[Nuapada, NUAPADA]"
9,Rajasthan,deeg,"[Deeg , Deeg]"
